# Chapter 10: Robustness

## Item 80: Take Advantage of Each Block in `try/except/else/finally`

In [55]:
def erics_cool_file_reader(filename):
    print("Step 1: lets gooo")
    handle = open(filename)
    try:
        print("Step 2: read the file")
        return handle.read()
    finally:
        print("Step 3: close the file")
        handle.close()


# This is essentially what `with open` is doing
erics_cool_file_reader("packages.csv")

Step 1: lets gooo
Step 2: read the file
Step 3: close the file


'Sydney, truck, 25\nMelbourne, boat, 6\nBrisbane, plane, 12\nPerth, road train, 90\nAdelaide, truck, 17\n'

In [56]:
# THE ORDER OF EXECUTION

throwaway_txt = "throwaway.txt"
with open(throwaway_txt, "wb") as f:
    f.write(b"\xf1\xf2\xf3\xf4")

data = erics_cool_file_reader("foop")

Step 1: lets gooo


FileNotFoundError: [Errno 2] No such file or directory: 'foop'

Note the order of operations. An exception did occur, but "finally" happened before the exception was propagated.

So-called finally! Its a conspiracy!!!

Bonus quiz: Why did the author not put `open` in the `try/finally` block?

In [57]:
# Else blocks
# These run when an exception didn't happen
import json


def get_some_json(data, key):
    try:
        print("READING")
        res = json.loads(data)
    except ValueError:
        print("WHOOPS")
        raise  # Author raises a KeyError, possibly for the caller's benefit?
    else:
        print("Looking!")
        return res[key]


assert get_some_json('{"foo": "bar"}', "foo") == "bar"

READING
Looking!


In [58]:
# We're only catching errors with loading json, propagate out the rest
get_some_json('{"foo": "bar"}', "baz")

READING
Looking!


KeyError: 'baz'

In [59]:
# Some bad json
get_some_json('{"foo" "bar"}', "foo")

READING
WHOOPS


JSONDecodeError: Expecting ':' delimiter: line 1 column 8 (char 7)

In [60]:
# PUT THEM ALL TOGETHER

UNDEFINED = object()


def divider(path):
    print("Open er up")
    handle = open(path, "r+")

    try:
        print("Read!!!")
        data = handle.read()

        print("Parse!!!")
        data = json.loads(data)

        print("Divide!!!")
        value = data["numerator"] / data["denominator"]
    except ZeroDivisionError:
        print("File not found")
        return UNDEFINED
    else:
        print("Calculate!!!")
        data["result"] = value
        result = json.dumps(data)
        handle.seek(0)
        handle.write(result)
        return value
    finally:
        print("Close!!!")
        handle.close()

In [61]:
# happy path
temp_path = "random_data.json"

with open(temp_path, "w+") as f:
    f.write('{"numerator": 1, "denominator": 10}')

assert divider(temp_path) == 0.1

with open(temp_path, "r+") as f:
    print(f.read())

Open er up
Read!!!
Parse!!!
Divide!!!
Calculate!!!
Close!!!
{"numerator": 1, "denominator": 10, "result": 0.1}


In [62]:
# zero division path
temp_path = "random_data.json"

with open(temp_path, "w+") as f:
    f.write('{"numerator": 1, "denominator": 0}')

assert divider(temp_path) is UNDEFINED

with open(temp_path, "r+") as f:
    print(f.read())

Open er up
Read!!!
Parse!!!
Divide!!!
File not found
Close!!!
{"numerator": 1, "denominator": 0}


In [63]:
# Bad data path
temp_path = "random_data.json"

with open(temp_path, "w+") as f:
    f.write('{"numerator": 1 lol what}')

assert divider(temp_path) == 0.1

with open(temp_path, "r+") as f:
    print(f.read())

Open er up
Read!!!
Parse!!!
Close!!!


JSONDecodeError: Expecting ',' delimiter: line 1 column 17 (char 16)

## Item 81: `assert` Internal Assumptions and `raise` Missed Expectations
Note: several linters will warn you about using `assert`

In [64]:
list_a = [1, 2, 3]
assert list_a, "MUST NOT BE EMPTY!!!"

list_b = []
assert list_b, "MUST NOT BE EMPTY!!!"

AssertionError: MUST NOT BE EMPTY!!!

In [65]:
# there's also raise
class EmptyError(Exception):
    pass


list_c = []
if not list_c:
    raise EmptyError("list_c is empty")

EmptyError: list_c is empty

In [66]:
try:
    raise EmptyError("raised it")
except EmptyError as e:
    print(f"caught {e}")

caught raised it


Note that `assert` raises `AssertionError`

So, when to use what?

### `raises`
This is considered part of a function's interface. The caller is expected to do something with it

### `assert`
This is not supposed to be part of the interface. It's used to verify assumptions in an implementation. It helps new readers of code understand that code better. Self-documenting. etc. Errors raised via assert can be treated as system problems, instead of "the caller made a mistake". "Our assumption was wrong"

In [67]:
class MovieRating:
    def __init__(self, max_rating):
        assert 0 <= max_rating <= 10, (
            "Rating must be between 0 and 10"
        )  # assumption: the caller validated the max rating
        self.max_rating = max_rating

    def rate(self, rating):
        if not 0 <= rating <= self.max_rating:
            raise ValueError(
                f"Rating must be between 0 and {self.max_rating}"
            )  # error: the caller gave too high a rating

In [68]:
rater = MovieRating(11)  # our code is bugged

AssertionError: Rating must be between 0 and 10

In [69]:
rater = MovieRating(10)
rater.rate(400)  # their code is bugged

ValueError: Rating must be between 0 and 10

Still fluffy?

That's okay, this is one of those "at your own discretion" areas.

My only warning around this (and the author warns this too) is to not catch `AssertionError`! Don't do it! It defeats the point. This includes not doing a high-level `except Exception`, because `AssertionError` _is_ an `Exception`.

### Things to Remember:
- The raise statement can be used to report expected error conditions back to the caller
- Exceptions raised by a function/method are part of its explicit interface
- The assert statement should be used to verify a programmers assumptions about code
- Dont catch/handle assertion errors! They indicate a bug!!!

### Side note: custom exceptions
stuff hes talking about here hints at something I _hope_ he talks about later (item 121): when building your own API, its usually better to make your own exception hierarchy, instead of using the builtins. That way you can have a top-level `except` for your module's code, without accidentally catching stuff you shouldn't (assertion errors!!!)

## Item 82: Consider `contextlib` and `with` Statements for Reusable `try/finally` Behavior

In [70]:
from threading import Lock
lock = Lock()

with lock:
    print("I'm running with a lock! Pretend theres threading code")

I'm running with a lock! Pretend theres threading code


In [71]:
# Basically what's happening:

lock.acquire()
try:
    print("I'm running with a lock! Pretend theres threading code")
finally:
    lock.release()

I'm running with a lock! Pretend theres threading code


Can save oneself repetitive code by using `with` instead of the frequent `try/finally`

In [72]:
# lets use it

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def my_func():
    logger.debug("some msg")
    logger.info("Info about stuff!")
    logger.debug("more msg")

my_func()

INFO:root:Info about stuff!


In [73]:
# Oh no no debug info! Maybe we set our app to that level. But what if we want a PARTICULAR area to always log its debug messages, but we don't care about other debug?

def my_func_2():
    current_level = logger.getEffectiveLevel()
    logger.setLevel(logging.DEBUG)
    logger.debug("some msg")
    logger.info("Info about stuff!")
    logger.debug("more msg")
    logger.setLevel(current_level)

my_func_2()

DEBUG:root:some msg
INFO:root:Info about stuff!
DEBUG:root:more msg


In [74]:
# Gross if we have to scale it up right?

from contextlib import contextmanager

@contextmanager
def debug_logging_on():
    current_level = logger.getEffectiveLevel()
    logger.setLevel(logging.DEBUG)
    try:
        yield
    finally:
        logger.setLevel(current_level)

In [75]:
# Behold
with debug_logging_on():
    my_func()

my_func()

DEBUG:root:some msg
INFO:root:Info about stuff!
DEBUG:root:more msg
INFO:root:Info about stuff!


We can get variables from the context with `as`.

```python
handle = open("my_file.txt", "w")
try:
    handle.write("cowabunga")
finally:
    handle.close()

with open("my_file.txt", "w") as handle:
    handle.write("cowabunga")
```

In [79]:
# do it ourselves

@contextmanager
def log_with(level, name):
    my_logger = logging.getLogger(name)
    current_level = my_logger.getEffectiveLevel()
    my_logger.setLevel(level)
    try:
        yield my_logger  # this is what is sent out on the `as` part
    finally:
        logger.setLevel(current_level)

with log_with(logging.DEBUG, "my_module") as loggeroni:
    loggeroni.debug("some msg")
    loggeroni.info("Info about stuff!")
    loggeroni.debug("more msg")
    logger.debug("The other logger! We shouldn't see this message")

DEBUG:my_module:some msg
INFO:my_module:Info about stuff!
DEBUG:my_module:more msg
DEBUG:root:The other logger! We shouldn't see this message
DEBUG:my_module:Shout into the void!!


In [78]:
# just for fun, lets get confusing
with log_with(logging.DEBUG, "my_module") as loggeroni:
    with debug_logging_on():
        loggeroni.debug("some msg")
        loggeroni.info("Info about stuff!")
        loggeroni.debug("more msg")
        logger.debug("The other logger! We shouldn't see this message")

DEBUG:my_module:some msg
INFO:my_module:Info about stuff!
DEBUG:my_module:more msg
DEBUG:root:The other logger! We shouldn't see this message


### Things to Remember
- `with` allows you some nice reusable try/finally behavior
- `contextlib`'s `contextmanager` makes it easy to build with-capable functions
- The value one yields from a context manager is what is provided to the `as` statement.

## Item 83: Always Make `try` Blocks as Short as Possible
It's so easy to make these huge.

Try to keep try blocks close to their error unit. Put only one source of **expected errors** in each try block

In [80]:
def shitty_func():
    raise ValueError("I'm a bad function")

def wacky_func():
    raise ValueError("el oh el i'm so crazy ;P")

try:
    shitty_func()
    wacky_func()
except ValueError as e:
    print("somebody goofed")

somebody goofed


But who goofed?!

Obviously we can reason about this example, but lets say these were real functions and it wasn't obvious if they'd throw or not.

In that case, we have to think more about "Who goofed"

In [81]:
try:
    shitty_func()
except ValueError as e:
    print("Shitty_Func definitely goofed!")
else:
    wacky_func() # will also goof, but thats not the point

Shitty_Func definitely goofed!


If you can, try to have as little in the try block as possible. Move other things to `else:` or their own try block

## Item 84: Beware of Exception Variables Disappearing

In a lot of places in python there are `blah as foo` statements, including exceptions: `except FooError as foo`. But unlike many others, the exception var does not live on past the exception block.

In [86]:
try:
    shitty_func()
except ValueError as e:
    print("It wont stop goofin!!!")

print(f"whats good {e=}")

It wont stop goofin!!!


NameError: name 'e' is not defined

In [87]:
# Oh that didn't work, well what about finally?

try:
    shitty_func()
except ValueError as e:
    print("It wont stop goofin!!!")
finally:
    print(f"whats good {e=}")

It wont stop goofin!!!


NameError: name 'e' is not defined

In [90]:
# if we DO need the variable outside the except block, its straight-forward:

goof_meter = "unknown goofs"

try:
    wacky_func()
except ValueError as e:
    goof_meter = "some goofs"
except AttributeError as e:
    goof_meter = "some goofs"
else:
    goof_meter = "no goofs"
finally:
    print(f"Approximate goof rate: {goof_meter}")

Approximate goof rate: some goofs


## Item 85: Beware of Catching the `Exception` Class
Oh boy, its the big one. Every linter in every language (that has exceptions) warns about this.

In [91]:
# Let's do some goof analysis

def load_data(raw_goofs):
    with open(raw_goofs) as f:
        return f.read()

def analyze_goofs(goofs):
    return goofs.count("goof")

def run_goof_report(path):
    data = load_data(path)
    goofs = analyze_goofs(data)
    return goofs

In [92]:
run_goof_report("daily_goofs-08-16-2025.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'daily_goofs-08-16-2025.txt'

In [93]:
# Aw dang it!
try:
    run_goof_report("daily_goofs-08-16-2025.txt")
except FileNotFoundError as e:
    print(f"Goof file not found: {e}")

Goof file not found: [Errno 2] No such file or directory: 'daily_goofs-08-16-2025.txt'


In [94]:
# solved! Oh, but now we also have other stuff to add
def run_goof_report(path):
    shitty_func()  # New behavior with its own error chain
    data = load_data(path)
    goofs = analyze_goofs(data)
    return goofs

try:
    run_goof_report("daily_goofs-08-16-2025.txt")
except Exception as e:  # Solved once and for all!!!
    print(f"Somebody really goofed: {e}")

Somebody really goofed: I'm a bad function


In [97]:
def run_goof_report(path):
    bug_in_ur_code()  # more new functionality introduced, shittier than before!
    shitty_func()
    data = load_data(path)
    goofs = analyze_goofs(data)
    return goofs

try:
    run_goof_report("daily_goofs-08-16-2025.txt")
except Exception as e:
    print(f"Somebody really goofed: {e}")

Somebody really goofed: name 'bug_in_ur_code' is not defined


The problem here, is nobody was goofin, this was a goofin bug in our goof code! This sort of thing should bubble up to the top for us to see! Not be handled and ignored, as it represents bad, unrecoverable state!

In [98]:
# Let's mitigate
try:
    run_goof_report("daily_goofs-08-16-2025.txt")
except Exception as e:
    print("Somebody really goofed:", type(e), e)

Somebody really goofed: <class 'NameError'> name 'bug_in_ur_code' is not defined
